# Storing FlexZBoost Data using Nested Pandas Dataframes

### Olivia's data path overrides
(Note, I've commented out the places where these were defined throughout the notebook)

In [ ]:
from pathlib import Path

data_dir = Path('/Users/orl/code/lincc/LINCC/Sarah/data')

trainFile = data_dir / 'lsst_cat_matched_nonan_train.hdf5'
testFile = data_dir / 'lsst_cat_matched_nonan_test.hdf5'

bpz_file = data_dir / '4bands/output_estimate_bpz.hdf5'
cmnn_file = data_dir / '4bands/output_estimate_cmnn.hdf5'
dnf_file = data_dir / '4bands/output_estimate_dnf.hdf5'
fzboost_file = data_dir / '4bands/output_estimate_fzboost.hdf5'
knn_file = data_dir / '4bands/output_estimate_knn.hdf5'
lephare_file = data_dir / '4bands/output_estimate_lephare.hdf5'
tpz_file = data_dir / '4bands/output_estimate_tpz.hdf5'

dp1_parquet_file = data_dir / 'dp1_ecdfs_edfs_sv95_sv37_v1.parquet'
dp1_subset_parquet_file = data_dir / '4bands/dp1_v29.0.0_gold_Rubin_SV_38_7_photoz_cat.parquet'

### Modules

In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tables_io
import qp
#import utils
import pickle
import pandas as pd
import desc_bpz
import lsdb
from rail.core.data import TableHandle, ModelHandle
from rail.utils.path_utils import RAILDIR
from rail.estimation.algos.bpz_lite import BPZliteInformer, BPZliteEstimator

from rail.core.data import TableHandle
from rail.core.stage import RailStage

from rail.evaluation.dist_to_dist_evaluator import DistToDistEvaluator
from rail.evaluation.dist_to_point_evaluator import DistToPointEvaluator
from rail.evaluation.point_to_point_evaluator import PointToPointEvaluator
from rail.evaluation.single_evaluator import SingleEvaluator
from rail.core.stage import RailStage
from rail.core.data import QPHandle, TableHandle, QPOrTableHandle

%matplotlib inline

/Users/orl/miniconda3/envs/rail/lib/python3.13/site-packages/ceci/__init__.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound


### Rail Stage Setup

In [4]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

### Loading Training and Test Data

In [5]:
from rail.utils.path_utils import find_rail_file
# trainFile = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/lsst_cat_matched_nonan_train.hdf5'
# testFile = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/lsst_cat_matched_nonan_test.hdf5'
training_data = DS.read_file("training_data", TableHandle, trainFile)
test_data = DS.read_file("test_data", TableHandle, testFile)

# Function to Create Nested DF to HATS

### Original versions: convert_ens, ens_to_df

In [6]:
def original_convert_ens(ens, algo):
    data_table = ens.build_tables()
    qp_type = data_table['meta']['pdf_name'][0].astype(str)
    converted_ensembles = {}
    converted_ensembles[f'{algo}_ens_{qp_type}'] = ens
    bins = np.linspace(0, 3, 301)
    quantile_lengths = [99, 20, 5]
    if qp_type == 'interp':
        converted_ensembles[f'{algo}_ens_{qp_type}_to_hist'] = ens.convert_to(qp.hist_gen, bins=bins)
        for qlen in quantile_lengths:
            quants = np.linspace(0.01, 0.99, qlen)
            var_name = f"{algo}_ens_{qp_type}_to_quant_{qlen}"
            try:
                print('here')
                converted_ensembles[var_name] = ens.convert_to(qp.quant_gen, quants=quants)
                print(f"Successfully created {var_name}")
            except Exception as e:
                print(f"Failed to create {var_name}: {e}")
        zmean = ens.mean()
        std = ens.std()
        converted_ensembles[f'{algo}_ens_{qp_type}_to_norm'] = qp.Ensemble(qp.stats.norm, data=dict(loc=zmean, scale=std))
        #converted_ensembles[f'ens_{qp_type}_to_mixmod'] = ens.convert_to(qp.mixmod_gen)
    if qp_type == 'norm':
        converted_ensembles[f'{algo}_ens_{qp_type}_to_hist'] = ens.convert_to(qp.hist_gen, bins=bins)
        for qlen in quantile_lengths:
            quants = np.linspace(0.01, 0.99, qlen)
            var_name = f"{algo}_ens_{qp_type}_to_quant_{qlen}"
            try:
                print('here')
                converted_ensembles[var_name] = ens.convert_to(qp.quant_gen, quants=quants)
                print(f"Successfully created {var_name}")
            except Exception as e:
                print(f"Failed to create {var_name}: {e}")
        converted_ensembles[f'{algo}_ens_{qp_type}_to_interp'] = ens.convert_to(qp.interp_gen, xvals=bins)
        #converted_ensembles[f'{algo}_ens_{qp_type}_to_mixmod'] = ens.convert_to(qp.mixmod_gen)
    if qp_type == 'mixmod':
        converted_ensembles[f'{algo}_ens_{qp_type}_to_hist'] = ens.convert_to(qp.hist_gen, bins=bins)
        for qlen in quantile_lengths:
            quants = np.linspace(0.01, 0.99, qlen)
            var_name = f"{algo}_ens_{qp_type}_to_quant_{qlen}"
            try:
                print('here')
                converted_ensembles[var_name] = ens.convert_to(qp.quant_gen, quants=quants)
                print(f"Successfully created {var_name}")
            except Exception as e:
                print(f"Failed to create {var_name}: {e}")
        #zmean = ens.mean()
        #std = ens.std()
        #converted_ensembles[f'{algo}_ens_{qp_type}_to_norm'] = qp.Ensemble(qp.stats.norm, data=dict(loc=zmean, scale=std))
        converted_ensembles[f'{algo}_ens_{qp_type}_to_interp'] = ens.convert_to(qp.interp_gen, xvals=bins)
    print("Done converting.")
    return converted_ensembles

In [ ]:
def original_ens_to_df(ens, algo):
    data_dict = {}
    converted_ensembles = original_convert_ens(ens, algo)
    for ensemble in converted_ensembles:
        data_dict[f'{ensemble}'] = original_convert_ens(converted_ensembles[f'{ensemble}'])
    df = pd.DataFrame(data_dict)
    return df

### New versions: convert_ens, ens_to_df

Note: this is kind of "exercise to the reader" territory!

I haven't checked that this fully converts everything we'd want in the way that we'd want--I've just checked that it looks approximately right and has the types we were looking for.

#### Updated Functions Summary

**Key Changes Made:**

1. **`convert_ens(ens, algo)`** - NEW VERSION
   - Same signature as `original_convert_ens(ens, algo)`
   - Performs all the same conversions (hist, quantiles, norm, interp)
   - Returns nested-pandas NestedFrames instead of regular DataFrames
   - Compatible with HATS/PyArrow conversion

2. **`ens_to_df(ens, algo)`** - NEW VERSION  
   - Works with the new `convert_ens` function
   - Creates DataFrames where each cell contains a dictionary of nested data
   - Compatible with HATS `df_to_hats()` function
   - Avoids the PyArrow conversion errors we were seeing before


In [20]:
import nested_pandas as npd
from nested_pandas.nestedframe import NestedFrame

def convert_ens_to_nested_frame(ens):
    """
    Convert a single ensemble to proper nested-pandas format
    """
    data_table = ens.build_tables()
    qp_type = data_table["meta"]["pdf_name"][0].astype(str)
    
    # Get the basic structure
    if qp_type == "norm" or qp_type == "mixmod":
        qp_dict = data_table["data"]
    elif qp_type == "interp":
        xvals = np.array(data_table["meta"]["xvals"])  # shape (301,)
        repeated_xvals = np.tile(xvals, (len(data_table["data"]["yvals"]), 1))
        xvals_dict = {"xvals": repeated_xvals}
        qp_dict = data_table["data"] | xvals_dict
    elif qp_type == "hist":
        bins = np.array(data_table["meta"]["bins"])
        repeated_bins = np.tile(bins, (len(data_table["data"]["pdfs"]), 1))
        bins_dict = {"bins": repeated_bins}
        qp_dict = data_table["data"] | bins_dict
    elif qp_type == "quant":
        quants = np.array(data_table["meta"]["quants"])  # shape (301,)
        repeated_quants = np.tile(quants, (len(data_table["data"]["locs"]), 1))
        quants_dict = {"quants": repeated_quants}
        qp_dict = data_table["data"] | quants_dict
    
    # Create flat data for nested-pandas
    flat_data = []
    
    first_key = next(iter(qp_dict))
    n_objects = len(qp_dict[first_key])
    
    for obj_idx in range(n_objects):
        for i in range(len(qp_dict[first_key][obj_idx])):  # iterate through array elements
            row_data = {'object_id': obj_idx}
            for key in qp_dict.keys():
                if key == "bins":
                    row_data[key] = qp_dict[key][obj_idx][i] if i < len(qp_dict[key][obj_idx]) - 1 else None
                else:
                    row_data[key] = qp_dict[key][obj_idx][i]
            if row_data.get(list(qp_dict.keys())[0]) is not None:  # Skip None values
                flat_data.append(row_data)
    
    # Convert to flat DataFrame
    flat_df = pd.DataFrame(flat_data)
    
    # Determine nested columns (all except object_id)
    nested_cols = [col for col in flat_df.columns if col != 'object_id']
    
    # Create NestedFrame
    nested_frame = NestedFrame.from_flat(
        flat_df,
        base_columns=[],  # No base columns except implicit index
        nested_columns=nested_cols,
        on='object_id',
        name=f'{qp_type}_data'
    )
    
    return nested_frame

def convert_ens(ens, algo):
    """
    Convert ensemble data with same signature as original_convert_ens
    This function matches the original signature and functionality, but returns 
    nested-pandas compatible DataFrames instead of storing separate DataFrames in cells.
    """
    data_table = ens.build_tables()
    qp_type = data_table['meta']['pdf_name'][0].astype(str)
    converted_ensembles = {}
    
    # Store the original ensemble as nested-pandas NestedFrame
    converted_ensembles[f'{algo}_ens_{qp_type}'] = convert_ens_to_nested_frame(ens)
    
    # Define conversion parameters (same as original)
    bins = np.linspace(0, 3, 301)
    quantile_lengths = [99, 20, 5]
    
    if qp_type == 'interp':
        # Convert to histogram
        hist_ens = ens.convert_to(qp.hist_gen, bins=bins)
        converted_ensembles[f'{algo}_ens_{qp_type}_to_hist'] = convert_ens_to_nested_frame(hist_ens)
        
        # Convert to quantiles
        for qlen in quantile_lengths:
            quants = np.linspace(0.01, 0.99, qlen)
            var_name = f"{algo}_ens_{qp_type}_to_quant_{qlen}"
            try:
                print('here')
                quant_ens = ens.convert_to(qp.quant_gen, quants=quants)
                converted_ensembles[var_name] = convert_ens_to_nested_frame(quant_ens)
                print(f"Successfully created {var_name}")
            except Exception as e:
                print(f"Failed to create {var_name}: {e}")
        
        # Convert to normal distribution
        zmean = ens.mean()
        std = ens.std()
        norm_ens = qp.Ensemble(qp.stats.norm, data=dict(loc=zmean, scale=std))
        converted_ensembles[f'{algo}_ens_{qp_type}_to_norm'] = convert_ens_to_nested_frame(norm_ens)
        
    elif qp_type == 'norm':
        # Convert to histogram
        hist_ens = ens.convert_to(qp.hist_gen, bins=bins)
        converted_ensembles[f'{algo}_ens_{qp_type}_to_hist'] = convert_ens_to_nested_frame(hist_ens)
        
        # Convert to quantiles
        for qlen in quantile_lengths:
            quants = np.linspace(0.01, 0.99, qlen)
            var_name = f"{algo}_ens_{qp_type}_to_quant_{qlen}"
            try:
                print('here')
                quant_ens = ens.convert_to(qp.quant_gen, quants=quants)
                converted_ensembles[var_name] = convert_ens_to_nested_frame(quant_ens)
                print(f"Successfully created {var_name}")
            except Exception as e:
                print(f"Failed to create {var_name}: {e}")
        
        # Convert to interpolation
        interp_ens = ens.convert_to(qp.interp_gen, xvals=bins)
        converted_ensembles[f'{algo}_ens_{qp_type}_to_interp'] = convert_ens_to_nested_frame(interp_ens)
        
    elif qp_type == 'mixmod':
        # Convert to histogram
        hist_ens = ens.convert_to(qp.hist_gen, bins=bins)
        converted_ensembles[f'{algo}_ens_{qp_type}_to_hist'] = convert_ens_to_nested_frame(hist_ens)
        
        # Convert to quantiles
        for qlen in quantile_lengths:
            quants = np.linspace(0.01, 0.99, qlen)
            var_name = f"{algo}_ens_{qp_type}_to_quant_{qlen}"
            try:
                print('here')
                quant_ens = ens.convert_to(qp.quant_gen, quants=quants)
                converted_ensembles[var_name] = convert_ens_to_nested_frame(quant_ens)
                print(f"Successfully created {var_name}")
            except Exception as e:
                print(f"Failed to create {var_name}: {e}")
        
        # Convert to interpolation
        interp_ens = ens.convert_to(qp.interp_gen, xvals=bins)
        converted_ensembles[f'{algo}_ens_{qp_type}_to_interp'] = convert_ens_to_nested_frame(interp_ens)
    
    print("Done converting.")
    return converted_ensembles

In [22]:
def ens_to_df(ens, algo):
    """Convert ensemble to DataFrame using nested-pandas format."""
    # Get the converted ensembles using the new convert_ens function
    converted_ensembles = convert_ens(ens, algo)
    
    # Create a base DataFrame with one row per object
    n_objects = len(next(iter(converted_ensembles.values())))
    df = pd.DataFrame(index=range(n_objects))
    
    # For each ensemble type, store the nested data in a way that HATS can handle
    for ensemble_name, nested_frame in converted_ensembles.items():
        series_data = []
        
        # The nested_frame should have nested columns containing the actual data
        if nested_frame.nested_columns:
            nested_col_name = nested_frame.nested_columns[0]
            
            # For each object (row in the base frame)
            for obj_id in range(n_objects):
                if obj_id < len(nested_frame):
                    # Get the nested data for this object
                    nested_data = nested_frame[nested_col_name].iloc[obj_id]
                    
                    # Convert the nested DataFrame to a dictionary
                    if hasattr(nested_data, 'to_dict'):
                        obj_dict = nested_data.to_dict('list')
                    elif isinstance(nested_data, dict):
                        obj_dict = nested_data
                    else:
                        # Try to convert to dict some other way
                        obj_dict = dict(nested_data) if nested_data is not None else {}
                    
                    series_data.append(obj_dict)
                else:
                    series_data.append({})
        else:
            # If no nested columns, create empty dicts
            series_data = [{}] * n_objects
        
        df[ensemble_name] = series_data
    
    return df

### Unchanged: convert_to_df

In [ ]:
def convert_to_df(ens):
    data_table = ens.build_tables()
    qp_type = data_table['meta']['pdf_name'][0].astype(str)
    if qp_type == 'norm' or qp_type == 'mixmod':
        qp_dict = data_table['data']
    if qp_type == 'interp':
        xvals = np.array(data_table['meta']['xvals'])  # shape (301,)
        repeated_xvals = np.tile(xvals, (len(data_table['data']['yvals']), 1))
        xvals_dict = {'xvals': repeated_xvals}
        qp_dict = data_table['data'] | xvals_dict
    if qp_type == 'hist':
        bins = np.array(data_table['meta']['bins'])
        repeated_bins = np.tile(bins, (len(data_table['data']['pdfs']), 1))
        bins_dict = {'bins': repeated_bins}
        qp_dict = data_table['data'] | bins_dict
    if qp_type == 'quant':
        quants = np.array(data_table['meta']['quants'])  # shape (301,)
        repeated_quants = np.tile(quants, (len(data_table['data']['locs']), 1))
        quants_dict = {'quants': repeated_quants}
        qp_dict = data_table['data'] | quants_dict
    first_key = next(iter(qp_dict))
    d = {}
    for i in range(len(qp_dict[f'{first_key}'])):
        temp_dict = {}
        for key in qp_dict.keys():
            if key == 'bins':
                temp_dict[key] = qp_dict[f'{key}'][i][:-1]
            else:
                temp_dict[key] = qp_dict[f'{key}'][i]
        d[i] = pd.DataFrame(temp_dict)
    return d


In [9]:
# df = pd.read_parquet('/Users/sarahpelesky/Downloads/dp1_ecdfs_edfs_sv95_sv37_v1.parquet')
# df_subset = pd.read_parquet('/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/sarah_data/4bands/dp1_v29.0.0_gold_Rubin_SV_38_7_photoz_cat.parquet')

df = pd.read_parquet(dp1_parquet_file)
df_subset = pd.read_parquet(dp1_subset_parquet_file)

new_df = pd.merge(df_subset, df, on='objectId', how='left', suffixes=('_subset', '_full'))
new_df

,objectId,knn_z_median_subset,knn_z_mode_subset,knn_z_err95_low_subset,knn_z_err95_high_subset,knn_z_err68_low_subset,knn_z_err68_high_subset,tpz_z_median_subset,tpz_z_mean_subset,tpz_z_mode_subset,...,dnf_z_err95_high_full,dnf_z_err68_low_full,dnf_z_err68_high_full,fzboost_z_median_full,fzboost_z_mean_full,fzboost_z_mode_full,fzboost_z_err95_low_full,fzboost_z_err95_high_full,fzboost_z_err68_low_full,fzboost_z_err68_high_full
0,648368263004160627,0.354946,0.35,0.279976,1.448482,0.323445,0.416408,0.929378,1.155020,0.76,...,1.930399,0.307735,1.813800,1.470184,1.688171,1.45,0.190596,2.899186,1.213509,2.837311
1,648368263004160629,0.870491,0.83,0.584325,1.347395,0.787742,1.300438,0.856566,0.850866,0.87,...,2.416718,0.805785,2.290113,0.909653,0.932351,0.89,0.776280,1.305105,0.846703,1.019736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169032,650020485383325799,0.912298,0.92,0.421746,4.847648,0.474989,1.215855,0.551099,0.642466,0.54,...,1.248661,0.441168,1.177997,0.463104,0.677022,0.46,0.032958,2.873164,0.172561,1.244638
169033,650020485383325824,0.884679,0.92,0.421952,1.344576,0.476390,0.976371,0.474240,0.572879,0.46,...,1.234473,0.452522,1.132565,0.427004,0.528916,0.41,0.006134,2.079242,0.339339,0.579889


In [10]:
# bpz_file = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/sarah_data/4bands/output_estimate_bpz.hdf5'
# cmnn_file = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/sarah_data/4bands/output_estimate_cmnn.hdf5'
# dnf_file = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/sarah_data/4bands/output_estimate_dnf.hdf5'
# fzboost_file = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/sarah_data/4bands/output_estimate_fzboost.hdf5'
# knn_file = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/sarah_data/4bands/output_estimate_knn.hdf5'
# lephare_file = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/sarah_data/4bands/output_estimate_lephare.hdf5'
# tpz_file = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/sarah_data/4bands/output_estimate_tpz.hdf5'

outputFiles_4bands = {
    "bpz_file": bpz_file,
    "cmnn_file": cmnn_file,
    "dnf_file": dnf_file,
    "fzboost_file": fzboost_file,
    "knn_file": knn_file,
    "lephare_file": lephare_file,
    "tpz_file": tpz_file,
}
algos = ["bpz", "cmnn", "dnf", "fzboost", "knn", "lephare", "tpz"]
all_algos = {}
i = 0
for file in outputFiles_4bands.keys():
    output_data = DS.read_file("output_data", QPHandle, outputFiles_4bands[file])
    df = ens_to_df_1(output_data.data, algos[i])
    all_algos[f"{algos[i]}"] = df
    i += 1

In [11]:
output_data = DS.read_file("output_data", QPHandle, bpz_file)
print(output_data.data.metadata)
print(output_data.data.objdata)

{'pdf_name': array([b'interp'], dtype='|S6'), 'pdf_version': array([0]), 'xvals': array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  , 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09,
       1.1 , 1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19, 1.2 ,
       1.21, 1.22, 1.23, 1.24, 1.25, 1.26, 1.27, 1.28, 1.29, 1.3 , 1.31,
       1.32, 1.33, 1.34, 1.35, 1.36, 1.37,

In [23]:
output_data = DS.read_file("output_data", QPHandle, bpz_file)
df = ens_to_df(output_data.data, 'bpz')

/Users/orl/miniconda3/envs/rail/lib/python3.13/site-packages/qp/parameterizations/hist/hist.py:140: RuntimeWarning: There are negative values in the pdfs for the distributions: [15124]
  warnings.warn(


here
Successfully created bpz_ens_interp_to_quant_99
here
Successfully created bpz_ens_interp_to_quant_99
here
Successfully created bpz_ens_interp_to_quant_20
here
Successfully created bpz_ens_interp_to_quant_20
here
Successfully created bpz_ens_interp_to_quant_5
Successfully created bpz_ens_interp_to_quant_5


/Users/orl/miniconda3/envs/rail/lib/python3.13/site-packages/scipy/stats/_distn_infrastructure.py:1460: RuntimeWarning: invalid value encountered in sqrt
  res = sqrt(self.stats(*args, **kwds))


Done converting.


In [24]:
df['coord_ra'] = new_df['coord_ra']
df['coord_dec'] = new_df['coord_dec']
df

,bpz_ens_interp,bpz_ens_interp_to_hist,bpz_ens_interp_to_quant_99,bpz_ens_interp_to_quant_20,bpz_ens_interp_to_quant_5,bpz_ens_interp_to_norm,coord_ra,coord_dec
0,"{'yvals': [0.0, 0.05513663978250978, 0.1068442...","{'pdfs': [0.02756831989125489, 0.0809904364631...","{'locs': [0.037283232580966996, 0.061513217592...","{'locs': [0.04591148831496429, 0.0615132175920...","{'locs': [0.05313888364411205, 0.0615132175920...","{'loc': [0.42742514450305613], 'scale': [0.358...",37.691623,7.384259
1,"{'yvals': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","{'pdfs': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","{'locs': [0.6537138438429267, 0.68143610109248...","{'locs': [0.6673293784597863, 0.68143610109248...","{'locs': [0.6761759467206362, 0.68143610109248...","{'loc': [0.9007582468804702], 'scale': [0.1842...",37.825903,7.384505
...,...,...,...,...,...,...,...,...
169032,"{'yvals': [0.0, 0.05950714207418124, 0.1165122...","{'pdfs': [0.02975357103709062, 0.0880097171943...","{'locs': [0.034157510332403385, 0.060015429903...","{'locs': [0.041979860352284334, 0.060015429903...","{'locs': [0.04960739182574118, 0.0600154299039...","{'loc': [0.5970112416211045], 'scale': [0.4137...",37.408291,7.891881
169033,"{'yvals': [0.0, 0.29417214903714967, 0.5238156...","{'pdfs': [0.14708607451857483, 0.4089939230504...","{'locs': [0.01411907719545967, 0.0271376218298...","{'locs': [0.017641523020463352, 0.027137621829...","{'locs': [0.02061429183617046, 0.0271376218298...","{'loc': [0.45371880592382235], 'scale': [0.440...",37.404037,7.901374


In [25]:
df.loc[0 , 'bpz_ens_interp']

{'yvals': [0.0,
  0.05513663978250978,
  0.10684423314386826,
  0.15640864660053766,
  0.20654985616512253,
  0.26225625237705447,
  0.31931387218935764,
  0.3831288712105078,
  0.45353125245991804,
  0.5340571085907977,
  0.6235984672456718,
  0.7164214650476612,
  0.8026977712166852,
  0.8760435255040967,
  0.9380668368360101,
  1.0049528660382478,
  1.0646002910447865,
  1.172033134746723,
  1.2828499710466372,
  1.395868368943043,
  1.5052564616890651,
  1.6230789427294106,
  1.7392600769751565,
  1.8602827913063202,
  1.9516848199584222,
  2.0274963202704064,
  2.0754369722542454,
  2.116002241318785,
  2.181142788061309,
  2.258167662243997,
  2.3527408418707236,
  2.4506616578943334,
  2.528964927802282,
  2.5933970774535107,
  2.6367541006172885,
  2.6733118496434782,
  2.6940921574089005,
  2.6967813327873156,
  2.6722567869591614,
  2.636663663523731,
  2.6028399922650123,
  2.574914234945079,
  2.5529654446116368,
  2.518274099796481,
  2.4692257583025894,
  2.39821716565199

In [26]:
frames = []
for frame in all_algos:
    frames.append(all_algos[f'{frame}'])
results = pd.concat(frames, axis=1)
results['coord_ra'] = new_df['coord_ra']
results['coord_dec'] = new_df['coord_dec']
results

,bpz_interp,cmnn_norm,dnf_interp,fzboost_interp,knn_mixmod,lephare_interp,tpz_interp,coord_ra,coord_dec
0,yvals xvals 0 0.000000 0.00 1 ...,loc scale 0 0.346 1.130511,yvals xvals 0 2.636724e-09 ...,yvals xvals 0 0.0 0.00 1 0.0...,weights stds means 0 0.180656 0...,yvals xvals 0 0.000000 0.00 1 ...,yvals xvals 0 0.000000 0.00498...,37.691623,7.384259
1,yvals xvals 0 0.0 0.00 1 0.0...,loc scale 0 0.822401 0.206878,yvals xvals 0 2.534018e-60 0...,yvals xvals 0 0.0 0.00 1 0.0...,weights stds means 0 0.219342 0...,yvals xvals 0 0.000000e+00 0...,yvals xvals 0 0.0 0.004983 1 ...,37.825903,7.384505
...,...,...,...,...,...,...,...,...,...
169032,yvals xvals 0 0.000000 0.00 1 ...,loc scale 0 0.5774 0.659871,yvals xvals 0 1.087945e-17 ...,yvals xvals 0 0.734918 0.00 1 ...,weights stds means 0 0.152417 0...,yvals xvals 0 0.000000 0.00 1 ...,yvals xvals 0 0.0 0.004983 1 ...,37.408291,7.891881
169033,yvals xvals 0 0.000000 0.00 1 ...,loc scale 0 2.3021 1.330341,yvals xvals 0 6.742047e-18 ...,yvals xvals 0 3.129272 0.00 1 ...,weights stds means 0 0.149318 0...,yvals xvals 0 0.000001 0.00 1 ...,yvals xvals 0 0.058431 0.00498...,37.404037,7.901374


In [27]:
cat = df_to_hats(df, 'from_dataframe')
cat

,bpz_ens_interp,bpz_ens_interp_to_hist,bpz_ens_interp_to_quant_99,bpz_ens_interp_to_quant_20,bpz_ens_interp_to_quant_5,bpz_ens_interp_to_norm,coord_ra,coord_dec
npartitions=2,,,,,,,,
"Order: 3, Pixel: 2","nested<xvals: [double], yvals: [double]>","nested<bins: [double], pdfs: [double]>","nested<locs: [double], quants: [double]>","nested<locs: [double], quants: [double]>","nested<locs: [double], quants: [double]>","nested<loc: [double], scale: [double]>",double[pyarrow],double[pyarrow]
"Order: 5, Pixel: 4471",...,...,...,...,...,...,...,...


In [28]:
def hats_to_qp(file_path):
    from_dataframe_catalog = lsdb.open_catalog(file_path)
    df = from_dataframe_catalog.compute()
    print(type(df['bpz_ens_interp'].iloc[0]))
    ensembles = {}
    for key in df.keys():
        parts = key.split('_')
        data_dict = {}
        if parts[-1] == 'interp':
            yvals = []
            for i in range(len(df[key])):
                print(df[key][9743840754662961])
                yvals.append(df[key][i]['yvals'])
            data_dict['yvals'] = yvals
            data_dict['xvals'] = df[key][0]['xvals']
            ensembles[key] = qp.Ensemble(qp.interp_gen, data=data_dict)
        if parts[-1] == 'mixmod':
            weights = []
            stds = []
            means = []
            for i in range(len(df[key])):
                weights.append(df[key][i]['weights'])
                stds.append(df[key][i]['stds'])
                means.append(df[key][i]['means'])
            data_dict['means'] = np.array(means)
            data_dict['stds'] = np.array(stds)
            data_dict['weights'] = np.array(weights)
            ensembles[key] = qp.Ensemble(qp.mixmod_gen, data=data_dict)
        if parts[-1] == 'norm':
            loc = []
            scale = []
            for i in range(len(df[key])):
                loc.append(df[key][i]['loc'])
                scale.append(df[key][i]['scale'])
            data_dict['loc'] = np.array(loc)
            data_dict['scale'] = np.array(scale)
            ensembles[key] = qp.Ensemble(qp.stats.norm, data=data_dict)
        if parts[-1] == 'hist':
            pdfs = []
            for i in range(len(df[key])):
                pdfs.append(df[key][i]['pdfs'])
            data_dict['pdfs'] = np.array(pdfs)
            data_dict['bins'] = np.linspace(0, 3, 301)
            ensembles[key] = qp.Ensemble(qp.hist_gen, data=data_dict)
        if parts[-1] == '99' or parts[-1] == '20' or parts[-1] == '5':
            locs = []
            for i in range(len(df[key])):
                locs.append(df[key][i]['locs'])
            data_dict['locs'] = np.array(locs)
            data_dict['quants'] = np.array(df[key][0]['quants'])
            ensembles[key] = qp.Ensemble(qp.quant_gen, data=data_dict)
    return ensembles

d = hats_to_qp('from_dataframe')

<class 'pandas.core.frame.DataFrame'>
     xvals     yvals
0     0.00  0.000000
1     0.01  0.055137
..     ...       ...
299   2.99  0.070059
300   3.00  0.071274

[301 rows x 2 columns]


KeyError: 0

In [ ]:
print(d['bpz_ens_interp'].metadata)
print(d['bpz_ens_interp'].objdata)

In [ ]:
import hats
from hats.pixel_math import HealpixPixel
import os

### Change this path!!!
catalog_dir = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/from_dataframe'

### ----------------
### You probably won't have to change anything from here.

catalog = hats.read_hats(catalog_dir)

info_frame = catalog.partition_info.as_dataframe()

for index, partition in info_frame.iterrows():
    file_name = result = hats.io.paths.pixel_catalog_file(
        catalog_dir, HealpixPixel(partition["Norder"], partition["Npix"])
    )
    info_frame.loc[index, "size_on_disk"] = os.path.getsize(file_name)

info_frame = info_frame.astype(int)
info_frame["gbs"] = info_frame["size_on_disk"] / (1024 * 1024 * 1024)

In [ ]:
print(f'healpix orders: {info_frame["Norder"].unique()}')
print(f'num partitions: {len(info_frame["Npix"])}')
print("------")
print(f'min size_on_disk: {info_frame["gbs"].min():.4f}')
print(f'max size_on_disk: {info_frame["gbs"].max():.4f}')
print(f'size_on_disk ratio: {info_frame["gbs"].max()/info_frame["gbs"].min():.4f}')
print(f'total size_on_disk: {info_frame["gbs"].sum():.4f}')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.hist(info_frame["gbs"])

bins = [0, 0.5, 1, 2, 100]
labels = ["small-ish", "sweet-spot", "big-ish", "too-big"]
hist = np.histogram(info_frame["gbs"], bins=bins)[0]
pcts = hist / len(info_frame)
for i in range(0, len(labels)):
    print(f"{labels[i]} \t: {hist[i]} \t({pcts[i]*100:.1f} %)")

In [ ]:
cat.plot_moc()

In [ ]:
cat.plot_pixels()

In [ ]:
%%time
outputFile = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/sarah_data/4bands/output_estimate_bpz.hdf5'
output_data = DS.read_file("output_data", QPHandle, outputFile)
bpz_df = df_to_hats(output_data.data, new_df)

In [ ]:
%%time
outputFile = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/sarah_data/4bands/output_estimate_gpz.hdf5'
output_data = DS.read_file("output_data", QPHandle, outputFile)
gpz_df = df_to_hats(output_data.data, new_df)

In [ ]:
%%time
outputFile = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/sarah_data/4bands/output_estimate_cmnn.hdf5'
output_data = DS.read_file("output_data", QPHandle, outputFile)
cmnn_df = df_to_hats(output_data.data, new_df)

In [ ]:
%%time
outputFile = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/sarah_data/4bands/output_estimate_dnf.hdf5'
output_data = DS.read_file("output_data", QPHandle, outputFile)
dnf_df = df_to_hats(output_data.data, new_df)

In [ ]:
%%time
outputFile = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/sarah_data/4bands/output_estimate_fzboost.hdf5'
output_data = DS.read_file("output_data", QPHandle, outputFile)
fzboost_df = df_to_hats(output_data.data, new_df)

In [ ]:
%%time
outputFile = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/sarah_data/4bands/output_estimate_knn.hdf5'
output_data = DS.read_file("output_data", QPHandle, outputFile)
knn_df = df_to_hats(output_data.data, test_data_df)

In [ ]:
knn_df.loc[0, 'ens_mixmod']

In [ ]:
%%time
outputFile = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/sarah_data/4bands/output_estimate_lephare.hdf5'
output_data = DS.read_file("output_data", QPHandle, outputFile)
lephare_df = df_to_hats(output_data.data, test_data_df)

In [ ]:
lephare_df.loc[0, 'ens_interp_to_norm']

In [ ]:
%%time
outputFile = '/Users/sarahpelesky/Desktop/LINCC Internship/RAIL/Projects (RAIL)/sarah_data/4bands/output_estimate_tpz.hdf5'
output_data = DS.read_file("output_data", QPHandle, outputFile)
tpz_df = df_to_hats(output_data.data, test_data_df)

In [ ]:
tpz_df.loc[0]

# Histograms

In [ ]:
bands = ['u_gaap1p0Mag',
    'g_gaap1p0Mag',
    'r_gaap1p0Mag',
    'i_gaap1p0Mag',
    'z_gaap1p0Mag',
    'y_gaap1p0Mag']
converted_ensembles = convert_ens(output_data.data)
for band in bands:
    plt.hist2d(test_data.data[band], converted_ensembles['ens_interp'].mean().flatten(), bins=25)
    plt.xlabel(band)
    plt.ylabel('Redshift Mean')
    plt.ylim(0, 3)
    plt.colorbar()
    plt.show()

In [ ]:
def plot_dist(ens, plotname):
    expdfids = [1075, 1251, 1322, 2567]
    fig, axs = plt.subplots(4, 1, figsize=(12,10))
    for i, xx in enumerate(expdfids):
        ztrue = test_data()['redshift'][xx]
        if ztrue <= 3:
            axs[i].set_xlim(0,3)
            zgrid = np.linspace(0, 3, 301)
        else:
            axs[i].set_xlim(0,ztrue+0.1)
            zgrid = np.linspace(0, 3, 301)
        yvals = ens[xx].pdf(zgrid)
        ymax = np.nanmax(yvals)
        axs[i].plot([ztrue,ztrue], [0,ymax], "--", color = "black")
        ens[xx].plot_native(axes=axs[i])
    axs[3].set_xlabel("redshift", fontsize=15)
    #plt.savefig(plotname)

In [ ]:
plot_dist(converted_ensembles['ens_interp'], '')

In [ ]:
print(test_data.data['i_gaap1p0Mag'][1322])